# Relevance-ranked search

Let's return to the indexing of toy data, as we did in the tutorial on Boolean search. This new tutorial has also been inspired by course material by Filip Ginter in Turku.

Our documents now look slightly different:

In [1]:
documents = ["This is a silly silly silly example",
             "A better example",
             "Nothing to see here nor here nor here",
             "This is a great example and a long example too"]

We can index them as we did before:

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

cv = CountVectorizer(lowercase=True, binary=True)
binary_dense_matrix = cv.fit_transform(documents).T.todense()

print("Term-document matrix:\n")
print(binary_dense_matrix)

Term-document matrix:

[[0 0 0 1]
 [0 1 0 0]
 [1 1 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [1 0 0 1]
 [0 0 1 0]
 [0 0 0 1]]


Next, we'll remove the `binary=True` optional argument from the `CountVectorizer` constructor. The default value is `binary=False`. What change can we observe?

In [3]:
cv = CountVectorizer(lowercase=True)
dense_matrix = cv.fit_transform(documents).T.todense()

print("Term-document matrix:\n")
print(dense_matrix)

Term-document matrix:

[[0 0 0 1]
 [0 1 0 0]
 [1 1 0 2]
 [0 0 0 1]
 [0 0 3 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 2 0]
 [0 0 1 0]
 [0 0 1 0]
 [3 0 0 0]
 [1 0 0 1]
 [0 0 1 0]
 [0 0 0 1]]


If we run a query on the term "example", we get:

In [4]:
t2i = cv.vocabulary_  # shorter notation: t2i = term-to-index
print("Query: example")
print(dense_matrix[t2i["example"]])

Query: example
[[1 1 0 2]]


Instead of seeing *whether* a term occurs in a document, we now see *how many times* the term occurs in each document:

In [5]:
hits_list = np.array(dense_matrix[t2i["example"]])[0]

for i, nhits in enumerate(hits_list):
    print("Example occurs", nhits, "time(s) in document:", documents[i])

Example occurs 1 time(s) in document: This is a silly silly silly example
Example occurs 1 time(s) in document: A better example
Example occurs 0 time(s) in document: Nothing to see here nor here nor here
Example occurs 2 time(s) in document: This is a great example and a long example too


When the number and sizes of the documents grow, we may think that the more times a search term occurs in a document, the more relevant the document is. So, if we search for "example" in our toy document collection, the fourth document is most relevant (2 hits), the first and second documents come next (1 hit each) and the third document is irrelevant (0 hits).

If we have multiple search terms, we might think that the more times the search terms occur in total in the document, the more relevant the document is.

Note that the bit-wise logical operators `AND (&)` and `OR (|)` will not work properly anymore when our matrix contains word counts. The same applies to `NOT (1 - x)`.

Let's search for the most relevant document for the query *better example*:

In [6]:
print("Query: better example")
print("Hits of better:        ", dense_matrix[t2i["better"]])
print("Hits of example:       ", dense_matrix[t2i["example"]])
print("Hits of better example:", dense_matrix[t2i["better"]] + dense_matrix[t2i["example"]])

Query: better example
Hits of better:         [[0 1 0 0]]
Hits of example:        [[1 1 0 2]]
Hits of better example: [[1 2 0 2]]


We just added the hits together. This means that we did not search for the phrase "better example", nor did we search for "better" AND "example". What we did search for was some kind of "better" OR "example", in which the sum of the number of occurrences of "better" and "example" in a document determines the relevance of the document.

This means that the second document, which contains one occurrence each of "better" and "example" is as good a hit as the fourth document, which contains two occurrences of "example" and no occurrence of "better".

Let's do another query:

In [7]:
print("Query: silly example")
print("Hits of silly:        ", dense_matrix[t2i["silly"]])
print("Hits of example:      ", dense_matrix[t2i["example"]])
print("Hits of silly example:", dense_matrix[t2i["silly"]] + dense_matrix[t2i["example"]])

Query: silly example
Hits of silly:         [[3 0 0 0]]
Hits of example:       [[1 1 0 2]]
Hits of silly example: [[4 1 0 2]]


... and also rank (sort) the results by relevance. We leave out the document without a single hit:

In [8]:
hits_list = np.array(dense_matrix[t2i["silly"]] + dense_matrix[t2i["example"]])[0]
print("Hits:", hits_list)

nhits_and_doc_ids = [ (nhits, i) for i, nhits in enumerate(hits_list) if nhits > 0 ]
print("List of tuples (nhits, doc_idx) where nhits > 0:", nhits_and_doc_ids)

ranked_nhits_and_doc_ids = sorted(nhits_and_doc_ids, reverse=True)
print("Ranked (nhits, doc_idx) tuples:", ranked_nhits_and_doc_ids)

print("\nMatched the following documents, ranked highest relevance first:")
for nhits, i in ranked_nhits_and_doc_ids:
    print("Score of 'silly example' is", nhits, "in document:", documents[i])

Hits: [4 1 0 2]
List of tuples (nhits, doc_idx) where nhits > 0: [(4, 0), (1, 1), (2, 3)]
Ranked (nhits, doc_idx) tuples: [(4, 0), (2, 3), (1, 1)]

Matched the following documents, ranked highest relevance first:
Score of 'silly example' is 4 in document: This is a silly silly silly example
Score of 'silly example' is 2 in document: This is a great example and a long example too
Score of 'silly example' is 1 in document: A better example


## Tf-idf

As we may guess, pure word counts are not a good indicator of relevance. Frequently occurring words are not usually very interesting from the point of view of information content.

One approach to weight terms (words) by their relevance is to use *term frequency / inverse document frequency (tf-idf)* weighting. There is another [tutorial on tf-idf](tf-idf-gutenberg.ipynb) that illustrates how this weighting works.

As a matter of fact, scikit-learn library makes it easy for us to compute the tf-idf scores of terms in a document collection. Instead of the class `CountVectorizer` we can use `TfidfVectorizer`: 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

The TfidfVectorizer can be used with many different parameter values. One option is to count ordinary term frequencies. In this setup the resulting matrix should produce the same values as the one produced by the CountVectorizer:

In [10]:
# Parameters with which TfidfVectorizer does same thing as CountVectorizer
tfv1 = TfidfVectorizer(lowercase=True, sublinear_tf=False, use_idf=False, norm=None)
tf_matrix1 = tfv1.fit_transform(documents).T.todense()

print("TfidfVectorizer:")
print(tf_matrix1)

print("\nCountVectorizer:")
print(dense_matrix)

TfidfVectorizer:
[[ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 1.  1.  0.  2.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  3.  0.]
 [ 1.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  2.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 3.  0.  0.  0.]
 [ 1.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]

CountVectorizer:
[[0 0 0 1]
 [0 1 0 0]
 [1 1 0 2]
 [0 0 0 1]
 [0 0 3 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 2 0]
 [0 0 1 0]
 [0 0 1 0]
 [3 0 0 0]
 [1 0 0 1]
 [0 0 1 0]
 [0 0 0 1]]


The values are the same, except that the TfidfVectorizer produces floating-point values, whereas the CountVectorizer produces integer values.

Some useful parameters for the TfidfVectorizer are `sublinear_tf`, `use_idf` and `norm`.

`sublinear_tf=True` uses logarithmic word frequencies instead of linear ones. That is, if a term occurs 20 times, it is not 20 times more important than a term that occurs once:

In [11]:
tfv2 = TfidfVectorizer(lowercase=True, sublinear_tf=True, use_idf=False, norm=None)
tf_matrix2 = tfv2.fit_transform(documents).T.todense()

print("TfidfVectorizer (logarithmic term frequencies):")
print(tf_matrix2)

TfidfVectorizer (logarithmic term frequencies):
[[ 0.          0.          0.          1.        ]
 [ 0.          1.          0.          0.        ]
 [ 1.          1.          0.          1.69314718]
 [ 0.          0.          0.          1.        ]
 [ 0.          0.          2.09861229  0.        ]
 [ 1.          0.          0.          1.        ]
 [ 0.          0.          0.          1.        ]
 [ 0.          0.          1.69314718  0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 2.09861229  0.          0.          0.        ]
 [ 1.          0.          0.          1.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


`use_idf=True` factors in the inverse document frequencies. The more documents a term occurs in, the less relevant the term is, in general:

In [12]:
tfv3 = TfidfVectorizer(lowercase=True, sublinear_tf=True, use_idf=True, norm=None)
tf_matrix3 = tfv3.fit_transform(documents).T.todense()

print("TfidfVectorizer (logarithmic term frequencies and inverse document frequencies):")
print(tf_matrix3)

TfidfVectorizer (logarithmic term frequencies and inverse document frequencies):
[[ 0.          0.          0.          1.91629073]
 [ 0.          1.91629073  0.          0.        ]
 [ 1.22314355  1.22314355  0.          2.07096206]
 [ 0.          0.          0.          1.91629073]
 [ 0.          0.          4.02155128  0.        ]
 [ 1.51082562  0.          0.          1.51082562]
 [ 0.          0.          0.          1.91629073]
 [ 0.          0.          3.24456225  0.        ]
 [ 0.          0.          1.91629073  0.        ]
 [ 0.          0.          1.91629073  0.        ]
 [ 4.02155128  0.          0.          0.        ]
 [ 1.51082562  0.          0.          1.51082562]
 [ 0.          0.          1.91629073  0.        ]
 [ 0.          0.          0.          1.91629073]]


If additionally, we use the L2 norm `norm="l2"` we normalize all document vectors (columns) to have a (Euclidian) length of one:

In [13]:
tfv4 = TfidfVectorizer(lowercase=True, sublinear_tf=True, use_idf=True, norm="l2")
tf_matrix4 = tfv4.fit_transform(documents).T.todense()

print("TfidfVectorizer (logarithmic term frequencies and inverse document frequencies, normalized document vectors):")
print(tf_matrix4)

TfidfVectorizer (logarithmic term frequencies and inverse document frequencies, normalized document vectors):
[[ 0.          0.          0.          0.39494151]
 [ 0.          0.84292635  0.          0.        ]
 [ 0.25939836  0.53802897  0.          0.42681878]
 [ 0.          0.          0.          0.39494151]
 [ 0.          0.          0.65482842  0.        ]
 [ 0.32040859  0.          0.          0.31137642]
 [ 0.          0.          0.          0.39494151]
 [ 0.          0.          0.52831145  0.        ]
 [ 0.          0.          0.31202925  0.        ]
 [ 0.          0.          0.31202925  0.        ]
 [ 0.85287113  0.          0.          0.        ]
 [ 0.32040859  0.          0.          0.31137642]
 [ 0.          0.          0.31202925  0.        ]
 [ 0.          0.          0.          0.39494151]]


We can search the index in the same way as above, even if we use tf-idf weighting:

In [14]:
print("Query: silly example")
print("Hits of silly:        ", tf_matrix4[t2i["silly"]])
print("Hits of example:      ", tf_matrix4[t2i["example"]])
print("Hits of silly example:", tf_matrix4[t2i["silly"]] + tf_matrix4[t2i["example"]])

Query: silly example
Hits of silly:         [[ 0.85287113  0.          0.          0.        ]]
Hits of example:       [[ 0.25939836  0.53802897  0.          0.42681878]]
Hits of silly example: [[ 1.11226949  0.53802897  0.          0.42681878]]


... and we can rank the documents using the tf-idf scores:

In [15]:
hits_list4 = np.array(tf_matrix4[t2i["silly"]] + tf_matrix4[t2i["example"]])[0]
print("Hits:", hits_list4)

hits_and_doc_ids = [ (hits, i) for i, hits in enumerate(hits_list4) if hits > 0 ]
print("List of tuples (hits, doc_idx) where hits > 0:", hits_and_doc_ids)

ranked_hits_and_doc_ids = sorted(hits_and_doc_ids, reverse=True)
print("Ranked (hits, doc_idx) tuples:", ranked_hits_and_doc_ids)

print("\nMatched the following documents, ranked highest relevance first:")
for hits, i in ranked_hits_and_doc_ids:
    print("Score of 'silly example' is {:.4f} in document: {:s}".format(hits, documents[i]))

Hits: [ 1.11226949  0.53802897  0.          0.42681878]
List of tuples (hits, doc_idx) where hits > 0: [(1.1122694945914164, 0), (0.53802896910335729, 1), (0.42681878177600086, 3)]
Ranked (hits, doc_idx) tuples: [(1.1122694945914164, 0), (0.53802896910335729, 1), (0.42681878177600086, 3)]

Matched the following documents, ranked highest relevance first:
Score of 'silly example' is 1.1123 in document: This is a silly silly silly example
Score of 'silly example' is 0.5380 in document: A better example
Score of 'silly example' is 0.4268 in document: This is a great example and a long example too


It makes sense that the document "This is a silly silly silly example" comes up on the top, but why does "A better example" now rank higher than "This is a great example and a long example too"? The former one contains only one occurrence of "example" whereas the latter one contains two. Can you figure out the reason?